## Scraping de dados de PR Canada
### BY: Alex Mazullo
#### asmazullo@gmail.com

In [162]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [163]:
#abrindo o crome automatizado e depois abrindo o site 
nvdor = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe')
time.sleep(4)
nvdor.get('https://myimmitracker.com/en/ca/trackers/pnp-paper-based-pr-applications-tracker')

#aqui existe uma configuracao do site para detectar robos, entao tem que fazer o login manualmente

C:\Users\Alex Mazullo\AppData\Local\Temp\ipykernel_10628\1722797379.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  nvdor = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe')


In [164]:
#ele traz so algumas linhas da tabela com conteudo, as outras estao vazias.  
#temos que testar se elas tem a classe ag-cell-no-focus ag-cell ag-cell-not-inline-editing ag-cell-value
#outro problema, ele so traz os dados das colunas que sao mostradas entao
#teos que pegar os dados destas colunas e colocar em um dataframe
#depois temos que andar com a tabela para a direita com scroll e pegar o resto
#no final fazer um merge

#aqui pega a altura total
alturatotal = nvdor.execute_script('return document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollHeight')

#====================================
#INICIANDO A PRIMEIRA PARTE

#criando a tabela1  vazia
df = pd.DataFrame({'ID':[], 'Username':[], 'DataCriado':[], 'UltimaAtualizacao':[], \
                   'Noc':[], 'Nacionalidade':[], 'PaisResidencia':[], 'Stream':[], \
                   'SponsorProvince':[], 'CurrentStatus':[],'ReceivedDate':[], 'AORDate':[], 'MedicalResquestDate':[]})

idachados=[]
for i in range(0,int(1+alturatotal/600)):
    soup = BeautifulSoup(nvdor.page_source, 'lxml')
    linhas = soup.find_all('div', class_='ag-row-level-0')
    for linha in linhas:
        if (linha.find('div', class_='ag-cell-no-focus ag-cell ag-cell-not-inline-editing ag-cell-value')==None):
            pass
        elif (linha.get('row') in idachados):
            print('ja tinha esse id, nao será computado o caso do usuario: '+linha.find('div',{'colid':'username'}).text)
        else:
            print(f"computando o caso do usuario: {linha.find('div',{'colid':'username'}).text}, id nr{linha.get('row')}")
            v_id = linha.get('row')
            v_username=linha.find('div', {'colid':'username'}).text
            try:
                v_datacreated = linha.find('div', {'colid':'c_at'}).text
            except:
                v_datacreated=''

            try:
                v_ultimaatualiza = linha.find('div', {'colid':'updated'}).text
            except:
                v_ultimaatualiza=''

            try:
                v_noc = linha.find('div', {'colid':'xinim-kesur-kopym-piped-setyd-vufur-bofeh-hovyp-poxyx'}).text
            except:
                v_noc=''

            try:
                v_nacionalidade = linha.find('div', {'colid':'xodos-nymub-sovog-zolyr-kafit-finit-hyluk-cymek-bixax'}).text
            except:
                v_nacionalidade=''

            try:
                v_paisondemora = linha.find('div', {'colid':'xigac-zisus-kikah-vedes-vulys-remos-nupyg-myvyh-zixux'}).text
            except:
                v_paisondemora=''

            try:
                v_stream = linha.find('div', {'colid':'xiven-fyrut-tatyh-bubak-zidis-kevek-sicer-tylab-caxex'}).text
            except:
                v_stream =''

            try:
                v_provinciasponsor = linha.find('div', {'colid':'xusoz-pefor-budag-kudif-hokul-kycig-hopeh-molir-haxyx'}).text
            except:
                v_provinciasponsor = ''
            try:    
                v_currentstatus = linha.find('div', {'colid':'xopos-kybed-picys-supot-gukab-tetyl-luzyd-lekez-gixex'}).text
            except:
                v_currentstatus =''

            try:    
                v_dataaplicacaorecebida = linha.find('div', {'colid':'xulov-gobep-pepas-pitos-perut-lomut-lykem-vuric-loxix'}).text
            except:
                v_dataaplicacaorecebida =''

            try:    
                v_dateaor = linha.find('div', {'colid':'xuset-kavav-casez-nypek-sybet-synyg-nocan-tyzef-tyxux'}).text
            except:
                v_dateaor=''

            try:    
                v_datemedicalrequest = linha.find('div', {'colid':'xudec-tupac-gafyg-veboz-hanis-tavot-getim-kobal-myxix'}).text
            except:
                v_datemedicalrequest = ''
               
            df.loc[len(df)]=(v_id,v_username,v_datacreated,v_ultimaatualiza,v_noc,v_nacionalidade,v_paisondemora,v_stream,v_provinciasponsor,v_currentstatus,v_dataaplicacaorecebida,v_dateaor,v_datemedicalrequest)
            
                   
            idachados.append(linha.get('row'))
    
    nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollTop='+str(600*i))
    time.sleep(3)
print('primeira tabela criada com sucesso, iniciando a segunda fase')

#=========================================================================================================
#INICIANDO A SEGUNDA PARTE
#criando a tabela2 
df2 = pd.DataFrame({'ID':[], 'MedicalPassedDate':[], 'BGSStatus':[], 'Backgroundlastdatechange':[], \
                    'QTDPessoas':[], 'VisaOffice':[], 'FileTransferDate':[], 'DecisionDate':[], 'PPR':[],\
                    'DocCheck':[]})

# voltando a tabela para a parte de cima e para a direita
nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollTop=0')
nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollLeft=2000')
time.sleep(2)



idachados=[]
for i in range(0,int(1+alturatotal/600)):
    soup = BeautifulSoup(nvdor.page_source, 'lxml')
    linhas = soup.find_all('div', class_='ag-row-level-0')
    for linha in linhas:
        if (linha.find('div', class_='ag-cell-no-focus ag-cell ag-cell-not-inline-editing ag-cell-value')==None):
            pass
        elif (linha.get('row') in idachados):
            print('ja tinha esse id, nao será computado o caso do usuario: '+linha.get('row'))
        else:
            print(f"computando o ID da linha: {linha.get('row')}")
            v_id = linha.get('row')
            try:
                v_MedicalPassedDate = linha.find('div', {'colid':'xepot-nucyl-vycon-nivid-micim-rigal-semag-fyzyz-tixyx'}).text
            except:
                v_MedicalPassedDate=''

            try:
                v_BGSStatus = linha.find('div', {'colid':'xufol-codoz-vaban-rahuk-sobuc-tumyn-kesev-tikus-naxex'}).text
            except:
                v_BGSStatus=''

            try:
                v_Backgroundlastdatechange = linha.find('div', {'colid':'xifab-nehib-zuram-hitag-tyvib-fepem-lorak-zudat-nixax'}).text
            except:
                v_Backgroundlastdatechange =''

            try:
                v_QTDPessoas = linha.find('div', {'colid':'xobik-zodog-fabit-gakyk-mopip-cirof-kacav-bykaz-ryxox'}).text
            except:
                v_QTDPessoas=''

            try:
                v_VisaOffice = linha.find('div', {'colid':'xisap-cuvoc-citod-rycyf-lybas-kusef-fyzez-hedip-dyxyx'}).text
            except:
                v_VisaOffice=''

            try:
                v_FileTransferDate = linha.find('div', {'colid':'xidoz-nesat-datec-hyped-devat-cucuk-pydav-sezez-zixux'}).text
            except:
                v_FileTransferDate =''

            try:
                v_DecisionDate = linha.find('div', {'colid':'xofev-dacyt-megal-pomyg-pygef-gabed-ricig-zyvim-hyxox'}).text
            except:
                v_DecisionDate = ''
            try:    
                v_PPR = linha.find('div', {'colid':'xufad-nuraz-bapaz-pinyt-gydoz-nenaf-kipah-rapac-texux'}).text
            except:
                v_PPR =''

            try:    
                v_DocCheck = linha.find('div', {'colid':'xubot-hatel-matec-vycid-fycel-gugyp-tesag-ludyd-lexux'}).text
            except:
                v_DocCheck =''

            
               
            df2.loc[len(df2)]=(v_id,v_MedicalPassedDate, v_BGSStatus,v_Backgroundlastdatechange,v_QTDPessoas,\
                               v_VisaOffice,v_FileTransferDate, v_DecisionDate, v_PPR, v_DocCheck )
            
                   
            idachados.append(linha.get('row'))
    
    nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollTop='+str(600*i))
    time.sleep(2)
print('===============================================segunda tabela criada com sucesso')


#=========================================================================================================
#INICIANDO A terceira e ultima parte so para pegar a coluna Aditional info
#criando a tabela3 
df3 = pd.DataFrame({'ID':[], 'Moreinfo':[]})

# voltando a tabela para a parte de cima e para a direita
nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollTop=0')
nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollLeft=4000')
time.sleep(2)



idachados=[]
for i in range(0,int(1+alturatotal/600)):
    soup = BeautifulSoup(nvdor.page_source, 'lxml')
    linhas = soup.find_all('div', class_='ag-row-level-0')
    for linha in linhas:
        if (linha.find('div', class_='ag-cell-no-focus ag-cell ag-cell-not-inline-editing ag-cell-value')==None):
            pass
        elif (linha.get('row') in idachados):
            print('ja tinha esse id, nao será computado o caso do usuario: '+linha.get('row'))
        else:
            print(f"computando o ID da linha: {linha.get('row')}")
            v_id = linha.get('row')
            try:
                v_Moreinfo = linha.find('div', {'colid':'xugaz-tuhuz-kamih-mosyg-faput-lirod-romuk-napev-moxyx'}).text
            except:
                v_Moreinfo=''

            df3.loc[len(df3)]=(v_id,v_Moreinfo)
            idachados.append(linha.get('row'))
    
    nvdor.execute_script('document.querySelector("#center > div > div.ag-body > div.ag-body-viewport-wrapper > div").scrollTop='+str(600*i))
    time.sleep(2)
print('===============================================terceira tabela criada com sucesso')
print('===============================================fim da captura de dados')

computando o caso do usuario: calicocat, id nr0
computando o caso do usuario: SCUSBU, id nr1
computando o caso do usuario: mabakar25274, id nr2
computando o caso do usuario: Suneth C Wijeratne, id nr3
computando o caso do usuario: bkpc1, id nr4
computando o caso do usuario: HaBellout, id nr5
computando o caso do usuario: allanim, id nr6
computando o caso do usuario: kuljitsingh, id nr7
computando o caso do usuario: nimsara, id nr8
computando o caso do usuario: ASHOO ARORA, id nr9
computando o caso do usuario: MMB, id nr10
computando o caso do usuario: radi10, id nr11
computando o caso do usuario: aneska05, id nr12
computando o caso do usuario: shdream, id nr13
computando o caso do usuario: Lilini, id nr14
computando o caso do usuario: oduttca, id nr15
computando o caso do usuario: X W, id nr16
computando o caso do usuario: Meatballz, id nr17
computando o caso do usuario: ImmXe, id nr18
computando o caso do usuario: K Ken, id nr19
computando o caso do usuario: danielmmm, id nr20
computa

ja tinha esse id, nao será computado o caso do usuario: joshkim
ja tinha esse id, nao será computado o caso do usuario: cherifbrook
ja tinha esse id, nao será computado o caso do usuario: YYLL11
ja tinha esse id, nao será computado o caso do usuario: munafbhai umatiya
ja tinha esse id, nao será computado o caso do usuario: Erzie
ja tinha esse id, nao será computado o caso do usuario: phkara
ja tinha esse id, nao será computado o caso do usuario: mbanwar
ja tinha esse id, nao será computado o caso do usuario: Jay666
ja tinha esse id, nao será computado o caso do usuario: thelostmarin
ja tinha esse id, nao será computado o caso do usuario: Alex466
ja tinha esse id, nao será computado o caso do usuario: Rvdm
ja tinha esse id, nao será computado o caso do usuario: mvfsilva
ja tinha esse id, nao será computado o caso do usuario: Renata Dantas
ja tinha esse id, nao será computado o caso do usuario: Thao
ja tinha esse id, nao será computado o caso do usuario: Rodrigo Antinarelli
ja tinha esse

ja tinha esse id, nao será computado o caso do usuario: Anna Qin
ja tinha esse id, nao será computado o caso do usuario: GrigoriiK
ja tinha esse id, nao será computado o caso do usuario: Ashar Ali
ja tinha esse id, nao será computado o caso do usuario: Gakki
ja tinha esse id, nao será computado o caso do usuario: rovic
ja tinha esse id, nao será computado o caso do usuario: ianlee1982
ja tinha esse id, nao será computado o caso do usuario: beeb1360
ja tinha esse id, nao será computado o caso do usuario: wennanM
ja tinha esse id, nao será computado o caso do usuario: chukchuk
ja tinha esse id, nao será computado o caso do usuario: Shamilam
ja tinha esse id, nao será computado o caso do usuario: sssspan
ja tinha esse id, nao será computado o caso do usuario: Akhil123
ja tinha esse id, nao será computado o caso do usuario: AandN
ja tinha esse id, nao será computado o caso do usuario: klein
ja tinha esse id, nao será computado o caso do usuario: mvbcruise
ja tinha esse id, nao será computa

ja tinha esse id, nao será computado o caso do usuario: baramsun2
ja tinha esse id, nao será computado o caso do usuario: rwang3
ja tinha esse id, nao será computado o caso do usuario: Disha Bhadauria
ja tinha esse id, nao será computado o caso do usuario: Chau Hoang
ja tinha esse id, nao será computado o caso do usuario: Beingravipatel
ja tinha esse id, nao será computado o caso do usuario: cfk2023
ja tinha esse id, nao será computado o caso do usuario: abhishekmahajan19
ja tinha esse id, nao será computado o caso do usuario: rshahul
ja tinha esse id, nao será computado o caso do usuario: 55michan
ja tinha esse id, nao será computado o caso do usuario: ashasha
ja tinha esse id, nao será computado o caso do usuario: jiolami2021
ja tinha esse id, nao será computado o caso do usuario: shiffi mehta
ja tinha esse id, nao será computado o caso do usuario: Pankaj3303
ja tinha esse id, nao será computado o caso do usuario: AMY123
ja tinha esse id, nao será computado o caso do usuario: AgnesWa

ja tinha esse id, nao será computado o caso do usuario: phupv56
ja tinha esse id, nao será computado o caso do usuario: sarigsns
ja tinha esse id, nao será computado o caso do usuario: Mary Rose Villa
ja tinha esse id, nao será computado o caso do usuario: RK2019
ja tinha esse id, nao será computado o caso do usuario: Prcandidate10
ja tinha esse id, nao será computado o caso do usuario: Ricky Shen
ja tinha esse id, nao será computado o caso do usuario: designjeon
ja tinha esse id, nao será computado o caso do usuario: Garry1w
ja tinha esse id, nao será computado o caso do usuario: DacNhan0212
ja tinha esse id, nao será computado o caso do usuario: Luxixi
ja tinha esse id, nao será computado o caso do usuario: buttar78
ja tinha esse id, nao será computado o caso do usuario: dosi
ja tinha esse id, nao será computado o caso do usuario: fharrys
ja tinha esse id, nao será computado o caso do usuario: Ruby12
ja tinha esse id, nao será computado o caso do usuario: leomedeiros
ja tinha esse id

...

ja tinha esse id, nao será computado o caso do usuario: 525
ja tinha esse id, nao será computado o caso do usuario: 526
ja tinha esse id, nao será computado o caso do usuario: 527
ja tinha esse id, nao será computado o caso do usuario: 528
ja tinha esse id, nao será computado o caso do usuario: 529
ja tinha esse id, nao será computado o caso do usuario: 530
ja tinha esse id, nao será computado o caso do usuario: 531
ja tinha esse id, nao será computado o caso do usuario: 532
ja tinha esse id, nao será computado o caso do usuario: 533
ja tinha esse id, nao será computado o caso do usuario: 534
ja tinha esse id, nao será computado o caso do usuario: 535
ja tinha esse id, nao será computado o caso do usuario: 536
ja tinha esse id, nao será computado o caso do usuario: 537
ja tinha esse id, nao será computado o caso do usuario: 538
ja tinha esse id, nao será computado o caso do usuario: 539
ja tinha esse id, nao será computado o caso do usuario: 540
ja tinha esse id, nao será computado o c

ja tinha esse id, nao será computado o caso do usuario: 525
ja tinha esse id, nao será computado o caso do usuario: 526
ja tinha esse id, nao será computado o caso do usuario: 527
ja tinha esse id, nao será computado o caso do usuario: 528
ja tinha esse id, nao será computado o caso do usuario: 529
ja tinha esse id, nao será computado o caso do usuario: 530
ja tinha esse id, nao será computado o caso do usuario: 531
ja tinha esse id, nao será computado o caso do usuario: 532
ja tinha esse id, nao será computado o caso do usuario: 533
ja tinha esse id, nao será computado o caso do usuario: 534
ja tinha esse id, nao será computado o caso do usuario: 535
ja tinha esse id, nao será computado o caso do usuario: 536
ja tinha esse id, nao será computado o caso do usuario: 537
ja tinha esse id, nao será computado o caso do usuario: 538
ja tinha esse id, nao será computado o caso do usuario: 539
ja tinha esse id, nao será computado o caso do usuario: 540
ja tinha esse id, nao será computado o c

ja tinha esse id, nao será computado o caso do usuario: 525
ja tinha esse id, nao será computado o caso do usuario: 526
ja tinha esse id, nao será computado o caso do usuario: 527
ja tinha esse id, nao será computado o caso do usuario: 528
ja tinha esse id, nao será computado o caso do usuario: 529
ja tinha esse id, nao será computado o caso do usuario: 530
ja tinha esse id, nao será computado o caso do usuario: 531
ja tinha esse id, nao será computado o caso do usuario: 532
ja tinha esse id, nao será computado o caso do usuario: 533
ja tinha esse id, nao será computado o caso do usuario: 534
ja tinha esse id, nao será computado o caso do usuario: 535
ja tinha esse id, nao será computado o caso do usuario: 536
ja tinha esse id, nao será computado o caso do usuario: 537
ja tinha esse id, nao será computado o caso do usuario: 538
ja tinha esse id, nao será computado o caso do usuario: 539
ja tinha esse id, nao será computado o caso do usuario: 540
ja tinha esse id, nao será computado o c

ja tinha esse id, nao será computado o caso do usuario: 525
ja tinha esse id, nao será computado o caso do usuario: 526
ja tinha esse id, nao será computado o caso do usuario: 527
ja tinha esse id, nao será computado o caso do usuario: 528
ja tinha esse id, nao será computado o caso do usuario: 529
ja tinha esse id, nao será computado o caso do usuario: 530
ja tinha esse id, nao será computado o caso do usuario: 531
ja tinha esse id, nao será computado o caso do usuario: 532
ja tinha esse id, nao será computado o caso do usuario: 533
ja tinha esse id, nao será computado o caso do usuario: 534
ja tinha esse id, nao será computado o caso do usuario: 535
ja tinha esse id, nao será computado o caso do usuario: 536
ja tinha esse id, nao será computado o caso do usuario: 537
ja tinha esse id, nao será computado o caso do usuario: 538
ja tinha esse id, nao será computado o caso do usuario: 539
ja tinha esse id, nao será computado o caso do usuario: 540
ja tinha esse id, nao será computado o c

ja tinha esse id, nao será computado o caso do usuario: 1490
ja tinha esse id, nao será computado o caso do usuario: 1491
ja tinha esse id, nao será computado o caso do usuario: 1492
ja tinha esse id, nao será computado o caso do usuario: 1493
ja tinha esse id, nao será computado o caso do usuario: 1494
ja tinha esse id, nao será computado o caso do usuario: 1495
ja tinha esse id, nao será computado o caso do usuario: 1496
ja tinha esse id, nao será computado o caso do usuario: 1497
ja tinha esse id, nao será computado o caso do usuario: 1498
ja tinha esse id, nao será computado o caso do usuario: 1499
ja tinha esse id, nao será computado o caso do usuario: 1500
ja tinha esse id, nao será computado o caso do usuario: 1501
ja tinha esse id, nao será computado o caso do usuario: 1502
ja tinha esse id, nao será computado o caso do usuario: 1503
ja tinha esse id, nao será computado o caso do usuario: 1504
ja tinha esse id, nao será computado o caso do usuario: 1505
ja tinha esse id, nao se

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1578 entries, 0 to 1577
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1578 non-null   object
 1   Username             1578 non-null   object
 2   DataCriado           1578 non-null   object
 3   UltimaAtualizacao    1578 non-null   object
 4   Noc                  1578 non-null   object
 5   Nacionalidade        1578 non-null   object
 6   PaisResidencia       1578 non-null   object
 7   Stream               1578 non-null   object
 8   SponsorProvince      1578 non-null   object
 9   CurrentStatus        1578 non-null   object
 10  ReceivedDate         1578 non-null   object
 11  AORDate              1578 non-null   object
 12  MedicalResquestDate  1578 non-null   object
dtypes: object(13)
memory usage: 172.6+ KB


In [173]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1578 entries, 0 to 1577
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        1578 non-null   object
 1   MedicalPassedDate         1578 non-null   object
 2   BGSStatus                 1578 non-null   object
 3   Backgroundlastdatechange  1578 non-null   object
 4   QTDPessoas                1578 non-null   object
 5   VisaOffice                1578 non-null   object
 6   FileTransferDate          1578 non-null   object
 7   DecisionDate              1578 non-null   object
 8   PPR                       1578 non-null   object
 9   DocCheck                  1578 non-null   object
dtypes: object(10)
memory usage: 135.6+ KB


In [174]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1578 entries, 0 to 1577
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        1578 non-null   object
 1   Moreinfo  1578 non-null   object
dtypes: object(2)
memory usage: 37.0+ KB


In [175]:
tabelaunica = df.set_index('ID').join(df2.set_index('ID'))
tabelaunica = tabelaunica.join(df3.set_index('ID'))

In [176]:
tabelaunica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1578 entries, 0 to 1577
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Username                  1578 non-null   object
 1   DataCriado                1578 non-null   object
 2   UltimaAtualizacao         1578 non-null   object
 3   Noc                       1578 non-null   object
 4   Nacionalidade             1578 non-null   object
 5   PaisResidencia            1578 non-null   object
 6   Stream                    1578 non-null   object
 7   SponsorProvince           1578 non-null   object
 8   CurrentStatus             1578 non-null   object
 9   ReceivedDate              1578 non-null   object
 10  AORDate                   1578 non-null   object
 11  MedicalResquestDate       1578 non-null   object
 12  MedicalPassedDate         1578 non-null   object
 13  BGSStatus                 1578 non-null   object
 14  Backgroundlastdatechange  157

In [177]:
tabelaunica.to_csv('tabela_dados_pr_versao2.csv')